# Working experiment

In [1]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [2]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 27 # Change the range so the process is faster

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = True # speeds up the procees but showing slighly worse results
resample_frequency = 50 # in Hz, original freq is 500Hz

In [3]:
from toolbox.methods import read_data
all_epochs, all_rawdata = read_data (task = 'main', resample=resample, amount_of_subjects=amount_of_subjects, resample_frequency=resample_frequency)

Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency after: 50.0
Frequency before: 500.0
Frequency 

In [8]:
from sklearn.svm import SVC
# from pyrcn.echo_state_network import ESNClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

def train_timepoints(X, y, verbose=False, display_roc=False, acc_only = False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    minority = np.unique(y_train,return_counts=True)[1][0]
    mayority = np.unique(y_train,return_counts=True)[1][1]
    # print(mayority/minority)
    class_weight = {
        0: 1.0,  
        1: mayority/minority
    }
    sample_weights = np.array([class_weight[label] for label in y_train])
    
    # clf = CatBoostClassifier(task_type = 'GPU')
    clf = RandomForestClassifier(random_state=0, n_jobs=-1)
    clf = RandomForestClassifier(n_jobs=-1)
    # clf = LinearSVC(random_state=0, loss="hinge") # Faster than Random Forest
    clf.fit(X_train, y_train,
            sample_weight=sample_weights
            )

    y_pred = clf.predict(X_test)    
    if verbose:
        print(classification_report(y_test, y_pred))
        print(np.unique(y_test, return_counts=True))
        print(np.unique(y_pred, return_counts=True))
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    # features = clf.feature_importances_
    if display_roc:
        # I think this is not working
        from sklearn.metrics import roc_curve
        from sklearn.metrics import RocCurveDisplay
        fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=clf.classes_[1])
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
        return roc_display
    # if np.unique(y_pred, return_counts=True)[1].shape[0] == 1:
    #     unique_pred_0 = 0
    #     unique_pred_1 = np.unique(y_pred, return_counts=True)[1][0]  
    # else:
    #     unique_pred_0 = np.unique(y_pred, return_counts=True)[1][0]
    #     unique_pred_1 = np.unique(y_pred, return_counts=True)[1][1]
    if acc_only:
        return accuracy
    return accuracy, f1, roc #, features

real data mean acc = 0.64

shuffled data mean acc = 0.58


# Mean acc for all subjects

In [11]:
'''30s per subject, 13min for 26 subjects '''

import random
subject_acc = [None]*amount_of_subjects
timepoint_accuracy = [None]*amount_of_subjects

bool_shuffled = False

for subj in range(amount_of_subjects-1):
    
    print('Working on subj: ', subj)
    X = all_rawdata[subj]['epoch_dat']
    y = np.where(all_rawdata[subj]['metadata']['r_map'] == 45, 1, 0)
    y2 = all_rawdata[subj]['metadata']['deci']
    y3 = [a ^ b for a, b in zip(y, y2)]
    if bool_shuffled: random.shuffle(y3)
    numT = X.shape[2]
    timepoint_accuracy[subj] = [None]*numT
    for timepoint in range(numT):
        # print(timepoint)
        X_training = X[:,:,timepoint]
        timepoint_accuracy[subj][timepoint] = train_timepoints(X_training, y3, verbose=False, acc_only=True)

numpy_timepoint_accuracy = np.array(timepoint_accuracy)



Working on subj:  0
Working on subj:  1
Working on subj:  2
Working on subj:  3
Working on subj:  4
Working on subj:  5
Working on subj:  6
Working on subj:  7
Working on subj:  8
Working on subj:  9
Working on subj:  10
Working on subj:  11
Working on subj:  12
Working on subj:  13
Working on subj:  14
Working on subj:  15
Working on subj:  16
Working on subj:  17
Working on subj:  18
Working on subj:  19
Working on subj:  20
Working on subj:  21
Working on subj:  22
Working on subj:  23
Working on subj:  24
Working on subj:  25


/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_64458/3318715416.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_timepoint_accuracy = np.array(timepoint_accuracy)
